https://www.datacamp.com/community/tutorials/recommender-systems-python

In [2]:
import pandas as pd
dir = !kaggle datasets download -d sootersaalu/amazon-top-50-bestselling-books-2009-2019

In [3]:
# using the code below to unzip a file in case if you need to do so.
#import zipfile
#with zipfile.ZipFile("file.zip","r") as zip_ref:
#   zip_ref.extractall("targetdir")

In [4]:
df =pd.read_csv('dir/bestsellers with categories.csv')

In [5]:
df.head(10)

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
5,A Dance with Dragons (A Song of Ice and Fire),George R. R. Martin,4.4,12643,11,2011,Fiction
6,A Game of Thrones / A Clash of Kings / A Storm...,George R. R. Martin,4.7,19735,30,2014,Fiction
7,A Gentleman in Moscow: A Novel,Amor Towles,4.7,19699,15,2017,Fiction
8,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,4.7,5983,3,2018,Non Fiction
9,A Man Called Ove: A Novel,Fredrik Backman,4.6,23848,8,2016,Fiction


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


In [7]:
df['Genre'].unique()

array(['Non Fiction', 'Fiction'], dtype=object)

In [8]:
# There are 248 authors
authors = df['Author']
authors.head()

0                    JJ Smith
1                Stephen King
2          Jordan B. Peterson
3               George Orwell
4    National Geographic Kids
Name: Author, dtype: object

In [43]:
# top 50 authors with the most books
top_50 = df['Author'].value_counts().head(70)
top_50

Jeff Kinney                           12
Rick Riordan                          11
Gary Chapman                          11
Suzanne Collins                       11
American Psychological Association    10
                                      ..
Amy Ramos                              2
Tara Westover                          2
Joel Fuhrman MD                        2
Dr. Steven R Gundry MD                 2
Michelle Obama                         2
Name: Author, Length: 70, dtype: int64

In [30]:
# Selecting authors with at least 5 written books
five_books_plus = top_50[top_50 > 4]
name_with_most_books = list(five_books_plus.index)
name_with_most_books

['Jeff Kinney',
 'Rick Riordan',
 'Gary Chapman',
 'Suzanne Collins',
 'American Psychological Association',
 'Gallup',
 'Dr. Seuss',
 'Rob Elliott',
 'Eric Carle',
 'Stephen R. Covey',
 "Bill O'Reilly",
 'Stephenie Meyer',
 'Dav Pilkey',
 'Don Miguel Ruiz',
 'The College Board',
 'E L James',
 'Sarah Young',
 'J.K. Rowling',
 'Stieg Larsson',
 'Harper Lee',
 'Dale Carnegie',
 'Roger Priddy',
 'John Green',
 'Laura Hillenbrand',
 'R. J. Palacio',
 'Patrick Lencioni',
 'Giles Andreae',
 'John Grisham']

In [64]:
top_author = df[df['User Rating'] > 4.6 ][['Author', 'User Rating']]
top_author.groupby('Author')[['User Rating', 'Author']].head()

,User Rating,Author
0,4.7,JJ Smith
2,4.7,Jordan B. Peterson
3,4.7,George Orwell
4,4.8,National Geographic Kids
6,4.7,George R. R. Martin
...,...,...
544,4.8,R. J. Palacio
546,4.7,Jen Sincero
547,4.7,Jen Sincero
548,4.7,Jen Sincero


In [32]:
df[df['Author'] == 'Rick Riordan']['User Rating'].mean()

4.7727272727272725

In [33]:
def auther_mean_rating(name):
    mean = df[df['Author'] == name]['User Rating'].mean()
    return mean

In [41]:
auther_mean_rating('Jeff Kinney')

4.8

In [65]:
def all_author_mean_rating(author_name):
    l = {}
    for i in author_name:
        avg = round(auther_mean_rating(i),2)
        l[i] = avg
    return l

dict = all_author_mean_rating(top_author['Author'])
        

In [71]:
{k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse = True)}

{'Bill Martin Jr.': 4.9,
 'Dav Pilkey': 4.9,
 'Sherri Duskey Rinker': 4.9,
 'Lin-Manuel Miranda': 4.9,
 'Sarah Young': 4.9,
 'Jill Twiss': 4.9,
 'Alice Schertle': 4.9,
 'Pete Souza': 4.9,
 'Rush Limbaugh': 4.9,
 'Nathan W. Pyle': 4.9,
 'Patrick Thorpe': 4.9,
 'Chip Gaines': 4.9,
 'Eric Carle': 4.9,
 'Emily Winfield Martin': 4.9,
 'Dr. Seuss': 4.88,
 'J. K. Rowling': 4.85,
 'Mark R. Levin': 4.85,
 'Brandon Stanton': 4.83,
 'National Geographic Kids': 4.8,
 'Ron Chernow': 4.8,
 'Michelle Obama': 4.8,
 'Atul Gawande': 4.8,
 'Jeff Kinney': 4.8,
 'David Goggins': 4.8,
 'Marjorie Sarnat': 4.8,
 'Brené Brown': 4.8,
 'Rod Campbell': 4.8,
 'Giles Andreae': 4.8,
 'Adam Mansbach': 4.8,
 'Margaret Wise Brown': 4.8,
 'Raina Telgemeier': 4.8,
 'Phil Robertson': 4.8,
 'Mitch Albom': 4.8,
 'Joanna Gaines': 4.8,
 'Hopscotch Girls': 4.8,
 'Ann Whitford Paul': 4.8,
 'Anthony Bourdain': 4.8,
 'Robert Munsch': 4.8,
 'Julia Child': 4.8,
 'Crispin Boyer': 4.8,
 'Amy Shields': 4.8,
 'Wizards RPG Team': 4.8,
